In [1]:
cd ~/Documents/Galvanize/mtg-capstone/src/

/Users/benjaminwalzer/Documents/Galvanize/mtg-capstone/src


In [2]:
import requests
import psycopg2
import os
import numpy as np
from deck_scraping import format_deck, ReflexiveDict, parse_card_string

In [3]:
card_dict = ReflexiveDict()

In [4]:
raw_deck_list = '''4 Sakura-Tribe Elder
                4 Primeval Titan
                2 Sweltering Suns
                4 Search for Tomorrow
                4 Scapeshift
                2 Hour of Promise
                4 Farseek
                2 Explore
                2 Summoner's Pact
                3 Lightning Bolt
                2 Prismatic Omen
                4 Wooded Foothills
                3 Windswept Heath
                4 Valakut, the Molten Pinnacle
                4 Stomping Ground
                3 Cinder Glade
                6 Mountain
                2 Forest
                1 Wood Elves
                Sideboard
                2 Obstinate Baloth
                2 Chameleon Colossus
                3 Relic of Progenitus
                1 Engineered Explosives
                2 Anger of the Gods
                1 Nature's Claim
                2 Beast Within
                1 Ancient Grudge
                1 Reclamation Sage
                '''

In [111]:
with open('/Users/benjaminwalzer/Downloads/cardstorm_test_kanji_4.txt', 'r') as f:
    deck_list = f.read()

In [112]:
deck_list = deck_list.split('\n')

In [113]:
all_cardstorm_ids = card_dict.get_cardstorm_ids()

In [114]:
def vectorize_deck(deck_list):
    deck_vectors = []
    for card_string in deck_list:
        parse_status, card_name, card_count = parse_card_string(card_string)
        deck_vectors.append((card_name.lower(), card_count))
        
    return deck_vectors

In [115]:
def deck_to_dict(deck_list):
    deck_dict = {}
    for card_string in deck_list:
        if card_string.startswith('S'):
            break
        if not card_string:
            break
        parse_status, card_name, card_count = parse_card_string(card_string)
        cardstorm_id = card_dict[card_name.lower()]
        deck_dict[cardstorm_id] = int(card_count)
        
    return deck_dict

In [116]:
deck_dict = deck_to_dict(deck_list)

In [117]:
card_dict.keys()

dict_keys(['abandoned sarcophagus', 1, 'abandon reason', 2, 'abattoir ghoul', 3, 'abbey griffin', 4, 'abbot of keral keep', 5, 'aberrant researcher', 6, 'abhorrent overlord', 7, 'abomination of gudul', 8, 'abrade', 9, 'abrupt decay', 10, 'absolver thrull', 11, 'absorb vis', 12, 'abstruse interference', 13, 'abuna acolyte', 14, "abuna's chant", 15, 'abundance', 16, 'abundant growth', 17, 'abundant maw', 18, 'abyssal nocturnus', 19, 'abyssal persecutor', 20, 'abyssal specter', 21, 'abzan advantage', 22, 'abzan ascendancy', 23, 'abzan banner', 24, 'abzan battle priest', 25, 'abzan beastmaster', 26, 'abzan charm', 27, 'abzan falconer', 28, 'abzan guide', 29, 'abzan kin-guard', 30, 'abzan runemark', 31, 'abzan skycaptain', 32, 'akki underling', 177, 'academy raider', 33, 'academy researchers', 34, 'academy ruins', 35, 'accomplished automaton', 36, 'accorder paladin', 37, "accorder's shield", 38, 'accursed horde', 39, 'accursed spirit', 40, 'accursed witch', 41, 'acidic slime', 42, 'acid-spe

In [118]:
# need to make a 11,438 x 1 vector of this deck list
deck_vector = []
for cardstorm_id in all_cardstorm_ids:
    if cardstorm_id in deck_dict:
        deck_vector.append(deck_dict[cardstorm_id])
    else:
        deck_vector.append(0)
deck_vector = np.array(deck_vector)

In [119]:
# retrieve the product_features matrix from the db

In [120]:
dbname = os.environ['CAPSTONE_DB_DBNAME']
host = os.environ['CAPSTONE_DB_HOST']
username = os.environ['CAPSTONE_DB_USERNAME']
password = os.environ['CAPSTONE_DB_PASSWORD']

In [121]:
conn = psycopg2.connect('dbname={} host={} user={} password={}'.format(dbname, host, username, password))
cursor = conn.cursor()

In [122]:
query = '''SELECT cardstorm_id, features
           FROM product_matrices
           WHERE date = (SELECT MAX(date) FROM product_matrices)
           ORDER BY cardstorm_id ASC'''

cursor.execute(query)

In [123]:
results = cursor.fetchall()

In [124]:
feature_matrix = []
for cardstorm_id, features in results:
    feature_matrix.append(features)
    
feature_matrix = np.array(feature_matrix)

In [125]:
feature_matrix.shape

(11348, 10)

In [126]:
result = np.linalg.lstsq(feature_matrix, deck_vector)

In [127]:
result[0]

array([-0.15642874,  0.06220244, -0.31110072,  0.00287821,  0.44539073,
        0.08936423,  0.10256757, -0.01984965, -0.10238489, -0.29790892])

In [128]:
result[1]

array([ 53.44402113])

In [129]:
recreated = np.dot(result[0], feature_matrix.T)

In [130]:
np.argsort(recreated - deck_vector)[::-1]

array([7021, 7200, 1957, ..., 4250, 3790, 9211])

In [131]:
all_cardstorm_ids = np.array(all_cardstorm_ids)

In [132]:
for idx, cardstorm_id in enumerate(all_cardstorm_ids[np.argsort(recreated - deck_vector)[::-1]]):
    print(card_dict[cardstorm_id])
    
    if idx == 9:
        break

path to exile
plains
cryptic command
flooded strand
celestial colonnade
ghost quarter
snapcaster mage
island
supreme verdict
spell snare
